In [9]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [10]:
!pip install transformers

In [11]:
!pip install torch

In [12]:
import pandas as pd
import random
import json
import numpy as np

In [13]:
def convert_input(data):
  for entry in data:
    entry["text"]=entry["text"].replace("LOCATION2","location - 2").replace("LOCATION1","location - 1")
    for opinion in entry["opinions"]:
      opinion["target_entity"]=opinion["target_entity"].replace("LOCATION2","location - 2").replace("LOCATION1","location - 1")
      opinion["aspect"]=opinion["aspect"].replace("transit-location","transit location")
  return data

In [17]:
sentihood_locations=["location - 1","location - 2"]
sentihood_aspects=["general","price","safety","transit location"]
sentihood_sentiments=["none","positive","negative"]
sentihood_label2id = {"none": 0, "positive": 1, "negative": 2}
id2label = {0: "None", 1: "Positive", 2: "Negative"}
label2id = {"None": 0, "Positive": 1, "Negative": 2}

In [18]:
def generate_sentiments_NLI_M(data):
    output = []
    for entry in data:
        id = entry["id"]
        original_sentence = entry["text"]
        for location in sentihood_locations:
            if location in original_sentence:
                for aspect in sentihood_aspects:
                    auxiliary_sentence = f"{location} - {aspect}"
                    label = "none"
                    for opinion in entry["opinions"]:
                        if opinion["target_entity"] == location and opinion["aspect"] == aspect:
                            if opinion["sentiment"] == "Positive":
                                label = "positive"
                            elif opinion["sentiment"] == "Negative":
                                label = "negative"
                    output.append([id, original_sentence, auxiliary_sentence, sentihood_label2id[label], label])
    loc1_rows = sorted([row for row in output if "location - 1" in row[2]], key=lambda el: el[0])
    loc2_rows = sorted([row for row in output if "location - 2" in row[2]], key=lambda el: el[0])
    output = [["id", "original_sentence", "auxiliary_sentence", "label_id", "label"]]
    output += loc1_rows + loc2_rows
    df = pd.DataFrame(output)
    return df

In [20]:
with open("../input/jsonfile/jsonvalidator.json","r") as f:
  data = json.loads(f.read())
  data = convert_input(data)
  df=generate_sentiments_NLI_M(data)
  df.to_csv("auxilary_NLIM.csv",index=False,header=False)

auxdata=pd.read_csv(r"./auxilary_NLIM.csv",index_col=0)
auxdata.head()


In [21]:
with open("../input/jsonfile/jsonvalidator_test.json","r") as f:
  data = json.loads(f.read())
  data = convert_input(data)
  df=generate_sentiments_NLI_M(data)
  df.to_csv("auxilary_NLIM_test.csv",index=False,header=False)
auxdata_test=pd.read_csv(r"./auxilary_NLIM_test.csv",index_col=0)
auxdata_test.head()

In [22]:
with open("../input/jsonfile/jsonvalidator_val.json","r") as f:
  data = json.loads(f.read())
  data = convert_input(data)
  df=generate_sentiments_NLI_M(data)
  df.to_csv("auxilary_NLIM_val.csv",index=False,header=False)
auxdata_val=pd.read_csv(r"./auxilary_NLIM_val.csv",index_col=0)
auxdata_val.head()

In [23]:
train_original_sentences = list(auxdata.iloc[:,0])
train_auxiliary_sentences = list(auxdata.iloc[:,1])
train_labels = list(auxdata.iloc[:,2])
test_original_sentences = list(auxdata_test.iloc[:,0])
test_auxiliary_sentences = list(auxdata_test.iloc[:,1])
test_labels = list(auxdata_test.iloc[:,2])
val_original_sentences = list(auxdata_val.iloc[:,0])
val_auxiliary_sentences = list(auxdata_val.iloc[:,1])
val_labels = list(auxdata_val.iloc[:,2])


In [24]:
from transformers import DistilBertTokenizerFast

tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

train_encodings = tokenizer(train_original_sentences, train_auxiliary_sentences, truncation=True, padding=True)
val_encodings = tokenizer(val_original_sentences,val_auxiliary_sentences, truncation=True, padding=True)
test_encodings = tokenizer(test_original_sentences,test_auxiliary_sentences, truncation=True, padding=True)

In [25]:
import torch

class ABSA_Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = ABSA_Dataset(train_encodings, train_labels)
val_dataset=ABSA_Dataset(val_encodings, val_labels)
test_dataset=ABSA_Dataset(test_encodings, test_labels)

# print(list(torch.utils.data.DataLoader(train_dataset, batch_size=2)))
# print(ABSA_Dataset()[0])

dataset = train_dataset
  
# get the first sample and unpack
# first_data = dataset[1]
# features = first_data
# print(features)
print(len(dataset[0]))

In [26]:
from scipy.special import softmax
def get_predictions(data):
    predicted_labels = []
    scores = []
   
    data = pd.read_csv(data, header=0).values.tolist()
    for row in data:
        
        predicted_labels.append(int(row[0]))
        scores.append([float(el) for el in row[1:]])
    return predicted_labels, scores

In [27]:
from sklearn import metrics

def compute_sentihood_aspect_strict_accuracy(test_labels, predicted_labels):
    correct_count = 0
    num_examples = len(test_labels) // 4
    for i in range(num_examples-4):
        if test_labels[i * 4] == predicted_labels[i * 4]\
                and test_labels[i * 4 + 1] == predicted_labels[i * 4 + 1]\
                and test_labels[i * 4 + 2] == predicted_labels[i * 4 + 2]\
                and test_labels[i * 4 + 3] == predicted_labels[i * 4 + 3]:
            correct_count += 1
    return correct_count / num_examples


def compute_sentihood_aspect_macro_F1(test_labels, predicted_labels):
    total_precision = 0
    total_recall = 0
    num_examples = len(test_labels) // 4
    count_examples_with_sentiments = 0
    for i in range(num_examples-4):
        test_aspects = set()
        predicted_aspects = set()
        for j in range(4):
            if test_labels[i * 4 + j] != 0:
                test_aspects.add(j)
            if predicted_labels[i * 4 + j] != 0:
                predicted_aspects.add(j)
        if len(test_aspects) == 0:
            continue
        intersection = test_aspects.intersection(predicted_aspects)
        if len(intersection) > 0:
            precision = len(intersection) / len(predicted_aspects)
            recall = len(intersection) / len(test_aspects)
        else:
            precision = 0
            recall = 0
        total_precision += precision
        total_recall += recall
        count_examples_with_sentiments += 1
    ma_P = total_precision / count_examples_with_sentiments
    ma_R = total_recall / count_examples_with_sentiments
    return (2 * ma_P * ma_R) / (ma_P + ma_R)


def compute_sentihood_aspect_macro_AUC(test_labels, scores):
    aspects_test_labels = [[] for _ in range(4)]
    aspects_none_scores = [[] for _ in range(4)]
    for i in range(len(test_labels)):
        if test_labels[i] != 0:
            new_label = 0
        else:
            new_label = 1   # For metrics.roc_auc_score you need to use the score of the maximum label, so "None" : 1
        aspects_test_labels[i % 4].append(new_label)
        aspects_none_scores[i % 4].append(scores[i][0])
    aspect_AUC = []
    for i in range(4):
        aspect_AUC.append(metrics.roc_auc_score(aspects_test_labels[i], aspects_none_scores[i]))
    aspect_macro_AUC = np.mean(aspect_AUC)
    return aspect_macro_AUC


def compute_sentihood_sentiment_classification_metrics(test_labels, scores):
    """Compute macro AUC and accuracy for sentiment classification ignoring "None" scores"""
    # Macro AUC
    sentiment_test_labels = [[] for _ in range(4)]  # One list for each aspect
    sentiment_negative_scores = [[] for _ in range(4)]
    sentiment_predicted_label = []
    sentiment_test_label = []   # One global list
    for i in range(len(test_labels)):
        if test_labels[i] != 0:
            new_test_label = test_labels[i] - 1  # "Positive": 0, "Negative": 1
            sentiment_test_label.append(new_test_label)
            new_negative_score = scores[i][2] / (scores[i][1] + scores[i][2])   # Prob. of "Negative" ignoring "None"
            if new_negative_score > 0.5:
                sentiment_predicted_label.append(1)
            else:
                sentiment_predicted_label.append(0)
            sentiment_test_labels[i % 4].append(new_test_label)
            sentiment_negative_scores[i % 4].append(new_negative_score)
    sentiment_AUC = []
    for i in range(4):
        sentiment_AUC.append(metrics.roc_auc_score(sentiment_test_labels[i], sentiment_negative_scores[i]))
    sentiment_macro_AUC = np.mean(sentiment_AUC)

    # Accuracy
    sentiment_accuracy = metrics.accuracy_score(sentiment_test_label, sentiment_predicted_label)

    return sentiment_macro_AUC, sentiment_accuracy


def compute_metrics(predictions):
        scores = [softmax(prediction) for prediction in predictions[0]]
        predicted_labels = [np.argmax(x) for x in scores]
        print(len(predicted_labels))
        data = np.insert(scores, 0, predicted_labels, axis=1)
        predicted_labels, scores = get_predictions(data)
        test_labels1 = test_labels
        metrics = {}
        metrics["strict_acc"] = compute_sentihood_aspect_strict_accuracy(test_labels1, predicted_labels)
        metrics["F1"] = compute_sentihood_aspect_macro_F1(test_labels1, predicted_labels)
        metrics["aspect_AUC"] = compute_sentihood_aspect_macro_AUC(test_labels1, scores)
        sentiment_macro_AUC, sentiment_accuracy = compute_sentihood_sentiment_classification_metrics(test_labels1, scores)
        metrics["sentiment_acc"] = sentiment_accuracy
        metrics["sentiment_AUC"] = sentiment_macro_AUC
        return metrics

In [28]:
from transformers import DistilBertForSequenceClassification , Trainer, TrainingArguments, DistilBertConfig

from transformers import logging
logging.set_verbosity_debug()


epochs = 4
batch_size = 24
num_steps = len(train_dataset) * epochs // batch_size
warmup_steps = num_steps // 10  # 10% of the training steps
save_steps = num_steps // epochs    # Save a checkpoint at the end of each epoch


training_args = TrainingArguments(
    output_dir = r"./",          
    num_train_epochs = epochs,              
    per_device_train_batch_size = batch_size,  
    per_device_eval_batch_size = batch_size,   
    warmup_steps = warmup_steps,   
    weight_decay = 0.01,               
    logging_dir = r"./",            
    logging_steps = 10,
    evaluation_strategy = 'epoch',
    learning_rate = 2e-5,
    save_steps = save_steps
)

config = DistilBertConfig.from_pretrained(
    'distilbert-base-uncased',
    architectures = ['DistilBertForSequenceClassification'],
    hidden_size = 768,
    num_hidden_layers = 6,
    num_attention_heads = 12,
    hidden_dropout_prob = 0.1,
    num_labels = 3
)    

load_finetuned_model = False
if not load_finetuned_model:
    model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', config=config)

    trainer = Trainer(
        model=model,                         
        args=training_args,                  
        train_dataset=train_dataset,
        eval_dataset=val_dataset,
        #compute_metrics=compute_metrics                     
    )
    trainer.train()

In [29]:
evaluation_result = trainer.evaluate(test_dataset)
print(evaluation_result)

In [30]:
import pandas as pd


results = trainer.predict(test_dataset)

scores = [softmax(prediction) for prediction in results.predictions]
predicted_labels = [np.argmax(x) for x in scores]
predicted_labels[0:21]


In [31]:
csv_output = np.insert(scores, 0, predicted_labels, axis=1)
df = pd.DataFrame(csv_output)
df[0] = df[0].astype("int")
header = ["predicted_label"]
for label in label2id.keys():
    header.append(label)
header

In [32]:
df.to_csv(r"./result.csv", index=False, header=header)

In [33]:
dataset=pd.read_csv(r"./result.csv")
dataset.head()

In [34]:
def get_predictions():
    predicted_labels = []
    scores = []
    data =  pd.read_csv(r"./result.csv", header=0).values.tolist()
    for row in data:
        predicted_labels.append(int(row[0]))
        scores.append([float(el) for el in row[1:]])
    return predicted_labels,scores
predicted_labels,scores = get_predictions()
print(predicted_labels[0:20])

In [35]:
def main(task="NLI_M", dataset_type="sentihood"):
    predicted_labels, scores = get_predictions()
    test_original_sentences = list(auxdata_test.iloc[:,0])
    test_auxiliary_sentences = list(auxdata_test.iloc[:,1])
    test_labels = list(auxdata_test.iloc[:,2])
    if dataset_type == "sentihood":
        sentihood_aspect_strict_acc = compute_sentihood_aspect_strict_accuracy(test_labels, predicted_labels)
        print(f"{task} Sentihood aspect strict accuracy: {sentihood_aspect_strict_acc}")
        sentihood_aspect_macro_F1 = compute_sentihood_aspect_macro_F1(test_labels, predicted_labels)
        print(f"{task} Sentihood aspect macro F1: {sentihood_aspect_macro_F1}")
        sentihood_aspect_macro_AUC = compute_sentihood_aspect_macro_AUC(test_labels, scores)
        print(f"{task} Sentihood aspect macro AUC: {sentihood_aspect_macro_AUC}")

        sentihood_sentiment_macro_AUC, sentihood_sentiment_accuracy = compute_sentihood_sentiment_classification_metrics(
            test_labels, scores)
        print(f"{task} Sentihood sentiment accuracy: {sentihood_sentiment_accuracy}")
        print(f"{task} Sentihood sentiment macro AUC: {sentihood_sentiment_macro_AUC}")

main("NLI_M", "sentihood")